##### Copyright 2022 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Devices

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/devices"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/devices.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq 

## 1. Validation basics

When you are looking to run an algorithm on a real quantum computer (not a simulated one), there are often many additional constraints placed on the circuits you would like to run. Qubit connectivity, algorithm layout and types of gates used in the circuit all become much more important. Cirq uses the abtract class `Device` to represent constraints of an actual quantum processor, an example implementation of a device can be seen in the `cirq_google.Sycamore` class:

In [ ]:
import cirq_google
import networkx as nx
my_device = cirq_google.Sycamore
my_device

All devices are capable of validing moments, operations and circuits with the `validate_***` method to verify if they would work on that device or not. You can check if the following operations work on the Sycamore device:

In [ ]:
op1 = cirq.X(cirq.GridQubit(7 ,7))

try:
  my_device.validate_operation(op1)
except Exception as e:
  print(e)

Above you used a qubit that wasn't on the device and was invalid. Most validate operations also take into account things like supported gates and connectivity as well:

In [ ]:
q1, q2, q3 = cirq.GridQubit(7, 4), cirq.GridQubit(7, 5), cirq.GridQubit(7, 6)
op1 = cirq.H(q1)
op2 = cirq_google.SYC(q1, q3)

try:
  my_device.validate_operation(op1)
except Exception as e:
  print(e)

try:
  my_device.validate_operation(op2)
except Exception as e:
  print(e)

These validation operations can also be used with full circuits:

In [ ]:
my_circuit = cirq.Circuit(
    cirq.PhasedXPowGate(phase_exponent=0.3)(q1),
    cirq.PhasedXPowGate(phase_exponent=0.3)(q2),
    cirq_google.SYC(q1, q2),
    cirq_google.SYC(q2, q3),
)
my_device.validate_circuit(my_circuit)

`my_circuit` satisfies all the device constraints and could be run on a Sycamore device.

## 2. Metadata features

Some devices will also expose additional information via the `metadata` property. Metadata is usually exposed via the an instance (or subclass instance) of the `cirq.DeviceMetadata` class. You can look at the metadata information of the Sycamore device with:

In [ ]:
metadata = my_device.metadata
metadata

The Sycamore device is a 2d grid device that exposes a `cirq.GridDeviceMetadata` with a uniform set of gates across all the qubits as well as a planar nearest neighbor connectivity graph. You can explore the properties below:

In [ ]:
type(metadata)

In [ ]:
issubclass(type(metadata), cirq.DeviceMetadata)

Some properties that are common to all devices and belong to the `cirq.DeviceMetadata` class include the set of qubits on the device (`qubit_set`) and the connectivity graph of those qubits (`nx_graph`)

In [ ]:
# Properties common to all `cirq.DeviceMetadata`
qubit_set = metadata.qubit_set
nx_graph = metadata.nx_graph
print(qubit_set)

Some properties are unique to the subclass. For `cirq.GridDeviceMetadata` you can explore the `gateset` (supported gates across all qubits) and the gate_durations (Dictionary of times that it takes to execute these gates on the device) fields:

In [ ]:
# Properties unique to `cirq.GridDeviceMetadata`
gateset = metadata.gateset
durations = metadata.gate_durations

print(gateset)

These metadata features can be useful when designing/building algorithms around certain device information in order to tailor them for that device.

## 3. The `cirq.Device` interface

For advanced users (such as vendors) it is also possible to implement your own Device with its own unique constraints and metadata information. Below you can implement our own fictituous device:

In [ ]:
class MyDevice(cirq.Device):
  """Five qubits on a line, supporting X/Y/Z and CZ between neighbors."""

  def __init__(self):
      self._qubits = set(cirq.LineQubit.range(5))
      self._supported_gates = cirq.Gateset(
          cirq.XPowGate, cirq.YPowGate, cirq.ZPowGate, cirq.CZPowGate
      )

  def validate_operation(self, operation):
      """Check to make sure `operation` is valid.

      `operation` must be on qubits found on the device
      and if it is a two qubit gate the qubits must be adjacent

      Raises:
        ValueError: if operation acts on qubits not found on the device.
        ValueError: if two qubit gates have non-local interactions.
        ValueError: if the operation is not in the supported gates.
      """
      if any(x not in self._qubits for x in operation.qubits):
        raise ValueError("Using qubits not found on device.")

      if len(operation.qubits) == 2:
          p, q = operation.qubits
          if not p.is_adjacent(q):
            raise ValueError('Non-local interaction: {}'.format(repr(operation)))

      if operation not in self._supported_gates:
        raise ValueError("Unsupported operation type.")
  
  def validate_circuit(self, circuit):
    """Check to make sure `circuit` is valid.

    Calls validate_operation on all operations as well as imposing
    a global limit on the total number of CZ gates.

    Raises:
      ValueError: if `validate_operation` raises for any operation in the
        circuit.
      ValueError: if there are more than 10 CZ gates in the entire circuit.
    """
    super().validate_circuit(circuit) # calls our `validate_operation`
    cz_count = sum(1 for mom in circuit for op in mom if len(op.qubits) == 2)
    if cz_count >= 10:
      raise ValueError("Too many total CZs")

  @property
  def metadata(self):
    """MyDevice GridDeviceMetadata."""
    # Since `MyDevice` is planar it is a good idea to subclass the
    # GridDeviceMetadata class to communicate additional device information to
    # the user.
    return cirq.GridDeviceMetadata(
        qubit_pairs=[(p, q) for p in self._qubits for q in self._qubits if p.is_adjacent(q)],
        gateset=self._supported_gates
    )

and use it to validate circuits:

In [ ]:
my_custom_device = MyDevice()

my_circuit = cirq.Circuit(
    cirq.X(cirq.LineQubit(0)),
    cirq.X(cirq.LineQubit(2)),
    cirq.X(cirq.LineQubit(4)),
    cirq.CZ(*cirq.LineQubit.range(2))
)
too_many_czs = cirq.Circuit(
    cirq.CZ(*cirq.LineQubit.range(2)) for _ in range(11)
)

# Valid for my_custom_device.
my_custom_device.validate_circuit(my_circuit)


# too_many_czs operation(s) are fine locally.
for moment in too_many_czs:
  for op in moment:
    my_custom_device.validate_operation(op)

# But the device has global constraints which the circuit does not meet:
try:
  my_custom_device.validate_circuit(too_many_czs)
except Exception as e:
  print(e)

By default calling `validate_circuit` calls `validate_moment` on all the moments which calls `validate_operation` on all the operations. Depending on the scoping of constraints the device has, certain less local constraints might be better placed in `validate_moment` and certain global constraints might belong in `validate_circuit`. In addition to this you can also add metadata options to your device. You can define a metadata subclass of `cirq.DeviceMetadata` or you can use an inbuilt metadata class like `cirq.GridDeviceMetadata`:

In [ ]:
my_metadata = my_custom_device.metadata

# Display device graph:
nx.draw(my_metadata.nx_graph)

Success! You have used Devices in Cirq to validate circuits for compatability when running as well as explored metadata information on the device and implemented your own boilerplate device.